On April 9th, US Senators Catherine Cortez Masto (NV-D) and Michael Bennet (CO-D) [introduced](https://www.cortezmasto.senate.gov/news/press-releases/cortez-masto-colleagues-introduce-bill-to-cut-taxes-for-working-americans/) the [Tax Cuts for Workers Act](https://www.cortezmasto.senate.gov/wp-content/uploads/2025/04/OTT25089.pdf), a bill that expands the Earned Income Tax Credit (EITC) for filers without qualifying children. This report describes the reform, illustrates the impact for a hypothetical household, and summarizes the nationwide impact using the PolicyEngine microsimulation model.

[See how the TCWA affects your household here.](https://policyengine.org/us/household?reform=81471&focus=intro&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps)

In [153]:
from IPython.display import Markdown
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Define PolicyEngine's color palette
BLACK = "#000000"
BLUE_LIGHT = "#D8E6F3"
BLUE_PRIMARY = "#2C6496"
DARK_BLUE_HOVER = "#1d3e5e"
DARK_GRAY = "#616161"
DARKEST_BLUE = "#0C1A27"
GRAY = "#808080"
LIGHT_GRAY = "#F2F2F2"
MEDIUM_DARK_GRAY = "#D2D2D2"
MEDIUM_LIGHT_GRAY = "#BDBDBD"
WHITE = "#FFFFFF"

## How the EITC and TCWA work

While the US provides the EITC to filers with and without qualifying children, it limits eligibility and reduces the amount for those without children. Such filers must be between ages 25 and 65, and the credit phases in at 7.65% of earnings until earnings reach $4,220 (in 2026). At $10,840 of adjusted gross income ($18,100 for joint filers), the credit is reduced with 7.65% of AGI.

In 2021, the US passed the American Rescue Plan Act (ARPA), which included an expansion to the EITC for filers without qualifying children. ARPA lowered the minimum age to 19 (24 for eligible students) and removed the maximum age. It also doubled the phase-in and phase-out rates to 15.3%, while increasing the earnings and AGI levels that governed the credit, thereby increasing the maximum credit value. It did not adjust the “married filing joint bonus”, an additional AGI level for joint filers before the credit starts phasing out. Read our report on the ARPA EITC [here](https://policyengine.org/us/research/restoring-arpa-eitc).

The TCWA restores ARPA’s EITC changes, while setting the dollar figures to their 2021 values starting in 2026 (rather than the values they would have been if ARPA were made permanent, due to inflation adjustments). Table 1 displays these parameter values under current law and TCWA as of 2026.


In [154]:
# Create a table for the household impacts
provisions = [
    "Minimum age",
    "Maximum age",
    "Phase-in rate",
    "Earnings at which the credit fully phases in",
    "Phase-out start",
    "Married filing joint bonus for phase-out start",
    "Phase-out rate",
]
current_law = [25, 65, "7.65%", "$8,677", "$10,840", "$7,260", "7.65%"]
tcwa = [19, "N/A", "15.3%", "$9,820", "$11,610", "$7,260", "15.3%"]


# Create DataFrame
df = pd.DataFrame(
    {"Provision": provisions, "Current Law": current_law, "TCWA": tcwa}
)


# Display as markdown table
Markdown(
    "**Table 1: EITC Parameters for Filers Without Qualifying Children (2026)**\n\n"
    + df.to_markdown(index=False)
)

**Table 1: EITC Parameters for Filers Without Qualifying Children (2026)**

| Provision                                      | Current Law   | TCWA    |
|:-----------------------------------------------|:--------------|:--------|
| Minimum age                                    | 25            | 19      |
| Maximum age                                    | 65            | N/A     |
| Phase-in rate                                  | 7.65%         | 15.3%   |
| Earnings at which the credit fully phases in   | $8,677        | $9,820  |
| Phase-out start                                | $10,840       | $11,610 |
| Married filing joint bonus for phase-out start | $7,260        | $7,260  |
| Phase-out rate                                 | 7.65%         | 15.3%   |

These changes result in the maximum credit value rising from $662 ($8,677 * 7.65%) to $1,502 ($9,820 * 15.3%).


In [155]:
# Create income values from 0 to 40,000 in equal steps
income_steps = 400  # Number of data points
# Generate the original income values
employment_income_original = [
    i * (40000 / (income_steps - 1)) for i in range(income_steps)
]
# Round the income values to the nearest 100
employment_income = [
    round(income / 100) * 100 for income in employment_income_original
]

# Use the exact values provided
blue_values = [
    0.0,
    16.82448,
    33.64896,
    50.473442,
    67.29792,
    84.12239,
    100.946884,
    117.771355,
    134.59584,
    151.4203,
    168.24478,
    185.06926,
    201.89377,
    218.71825,
    235.54271,
    252.36719,
    269.19168,
    286.01614,
    302.8406,
    319.6651,
    336.48956,
    353.3141,
    370.13852,
    386.96304,
    403.78754,
    420.61197,
    437.4365,
    454.26093,
    471.08542,
    487.90988,
    504.73438,
    521.5589,
    538.38336,
    555.2078,
    572.0323,
    588.8568,
    605.6812,
    622.50574,
    639.3302,
    656.15466,
    672.9791,
    689.8037,
    706.6282,
    723.4526,
    740.27704,
    757.10156,
    773.9261,
    790.75055,
    807.5751,
    824.3995,
    841.22394,
    858.0484,
    874.873,
    891.69745,
    908.52185,
    925.3464,
    942.17084,
    958.9953,
    975.81976,
    992.64435,
    1009.46875,
    1026.2932,
    1043.1178,
    1059.9423,
    1076.7667,
    1093.5911,
    1110.4156,
    1127.2401,
    1144.0646,
    1160.8892,
    1177.7136,
    1194.538,
    1211.3624,
    1228.187,
    1245.0115,
    1261.8359,
    1278.6604,
    1295.4849,
    1312.3093,
    1329.1338,
    1345.9583,
    1362.7828,
    1379.6074,
    1396.4318,
    1413.2563,
    1430.0808,
    1446.9052,
    1463.7297,
    1480.5541,
    1497.3787,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1495.2997,
    1478.4751,
    1461.6504,
    1444.8262,
    1428.0018,
    1411.1771,
    1394.3525,
    1377.5282,
    1360.7036,
    1343.8793,
    1327.0549,
    1310.23,
    1293.4058,
    1276.5814,
    1259.7568,
    1242.9325,
    1226.1079,
    1209.2832,
    1192.4589,
    1175.6345,
    1158.8097,
    1141.9854,
    1125.161,
    1108.3364,
    1091.5121,
    1074.6875,
    1057.8628,
    1041.0386,
    1024.2141,
    1007.3899,
    990.56494,
    973.7406,
    956.9163,
    940.0917,
    923.26733,
    906.44275,
    889.6181,
    872.79376,
    855.9695,
    839.14484,
    822.32025,
    805.4959,
    788.67126,
    771.847,
    755.02264,
    738.1977,
    721.3734,
    704.5491,
    687.7244,
    670.90015,
    654.0755,
    637.2509,
    620.4266,
    603.60223,
    586.77734,
    569.953,
    553.12866,
    536.3044,
    519.47974,
    502.6551,
    485.8308,
    469.00623,
    452.1819,
    435.35754,
    418.5326,
    401.70825,
    384.8839,
    368.05933,
    351.235,
    334.4104,
    317.58582,
    300.76147,
    283.93713,
    267.11255,
    250.28784,
    233.46362,
    216.63892,
    199.81458,
    182.99036,
    166.1654,
    149.34106,
    132.51672,
    115.69238,
    98.8678,
    82.04321,
    65.21887,
    48.394287,
    31.569946,
    14.745605,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]


grey_values = [
    0.0,
    8.41224,
    16.82448,
    25.236721,
    33.64896,
    42.061195,
    50.473442,
    58.885677,
    67.29792,
    75.71015,
    84.12239,
    92.53463,
    100.946884,
    109.35912,
    117.771355,
    126.18359,
    134.59584,
    143.00807,
    151.4203,
    159.83255,
    168.24478,
    176.65704,
    185.06926,
    193.48152,
    201.89377,
    210.30598,
    218.71825,
    227.13046,
    235.54271,
    243.95494,
    252.36719,
    260.77945,
    269.19168,
    277.6039,
    286.01614,
    294.4284,
    302.8406,
    311.25287,
    319.6651,
    328.07733,
    336.48956,
    344.90186,
    353.3141,
    361.7263,
    370.13852,
    378.55078,
    386.96304,
    395.37527,
    403.78754,
    412.19974,
    420.61197,
    429.0242,
    437.4365,
    445.84872,
    454.26093,
    462.6732,
    471.08542,
    479.49765,
    487.90988,
    496.32217,
    504.73438,
    513.1466,
    521.5589,
    529.9711,
    538.38336,
    546.79553,
    555.2078,
    563.62006,
    572.0323,
    580.4446,
    588.8568,
    597.269,
    605.6812,
    614.0935,
    622.50574,
    630.91797,
    639.3302,
    647.74243,
    656.15466,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    658.63043,
    650.21826,
    641.80597,
    633.3936,
    624.98145,
    616.56915,
    608.157,
    599.7448,
    591.3325,
    582.9202,
    574.50806,
    566.0959,
    557.6836,
    549.2713,
    540.85913,
    532.4468,
    524.03467,
    515.6225,
    507.21002,
    498.79785,
    490.38568,
    481.9734,
    473.56122,
    465.14893,
    456.7366,
    448.32446,
    439.9123,
    431.49982,
    423.08765,
    414.6755,
    406.26318,
    397.85104,
    389.43872,
    381.0264,
    372.61426,
    364.2021,
    355.78992,
    347.37747,
    338.9653,
    330.55316,
    322.14084,
    313.72867,
    305.31638,
    296.90405,
    288.49188,
    280.07974,
    271.66742,
    263.25513,
    254.84296,
    246.43063,
    238.0185,
    229.60632,
    221.19385,
    212.78171,
    204.36954,
    195.95721,
    187.54507,
    179.13275,
    170.72046,
    162.30829,
    153.89612,
    145.48364,
    137.07153,
    128.65936,
    120.24719,
    111.8349,
    103.42255,
    95.010376,
    86.59808,
    78.18591,
    69.77374,
    61.36133,
    52.949158,
    44.536987,
    36.124695,
    27.712524,
    19.300232,
    10.887878,
    2.475708,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]


# Create DataFrame for plotting
df = pd.DataFrame(
    {
        "Employment Income": employment_income,
        "Current Law": grey_values,
        "Tax Cuts for Workers Act": blue_values,
    }
)

# Reshape data for plotting with multiple lines
df_melted = df.melt(
    id_vars="Employment Income",
    value_vars=["Current Law", "Tax Cuts for Workers Act"],
    var_name="Scenario",
    value_name="EITC Amount",
)

# Create the figure
fig = px.line(
    df_melted,
    x="Employment Income",
    y="EITC Amount",
    color="Scenario",
    color_discrete_map={
        "Current Law": GRAY,
        "Tax Cuts for Workers Act": BLUE_PRIMARY,
    },
    hover_data={
        "Employment Income": ":$,.0f",
        "EITC Amount": ":$,.0f",
    }
).update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_title="Employment income",
    yaxis_title="EITC amount",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 50, "pad": 4},
    title = "Figure 2: Credit value for married joint filers under current law and the TCWA",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.8)",
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
    images=[
        {
            "source": "/logo512.png",
            "x": 1,
            "y": -0.2,
            "xref": "paper",
            "yref": "paper",
            "sizex": 0.1,
            "sizey": 0.1,
            "xanchor": "right",
            "yanchor": "bottom",
        }
    ],
)

# Update hover template
fig.update_traces(
    hovertemplate="<b>%{data.name}</b><br>" +
                  "Employment income: $%{x:,.0f}<br>" +
                  "EITC amount: $%{y:,.2f}<extra></extra>"
)

# Display the figure
Markdown(
    f"```plotly\n{fig.to_json()}\n```"
)

```plotly
{"data":[{"hovertemplate":"\u003cb\u003e%{data.name}\u003c\u002fb\u003e\u003cbr\u003eEmployment income: $%{x:,.0f}\u003cbr\u003eEITC amount: $%{y:,.2f}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Current Law","line":{"color":"#808080","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Current Law","orientation":"v","showlegend":true,"x":[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,5000,5100,5200,5300,5400,5500,5600,5700,5800,5900,6000,6100,6200,6300,6400,6500,6600,6700,6800,6900,7000,7100,7200,7300,7400,7500,7600,7700,7800,7900,8000,8100,8200,8300,8400,8500,8600,8700,8800,8900,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900,10000,10100,10200,10300,10400,10500,10600,10700,10800,10900,11000,11100,11200,11300,11400,11500,11600,11700,11800,11900,12000,12100,12200,12300,12400,12500,12600,12700,12800,12900,13000,13100,13200,13300,13400,13500,13600,13700,13800,13900,14000,14100,14200,14300,14400,14500,14600,14700,14800,14900,15000,15100,15200,15300,15400,15500,15600,15700,15800,15900,16000,16100,16200,16300,16400,16500,16600,16700,16800,16900,17000,17100,17200,17300,17400,17500,17600,17700,17800,17900,18000,18100,18200,18300,18400,18500,18600,18700,18800,18900,19000,19100,19200,19300,19400,19500,19600,19700,19800,19900,20100,20200,20300,20400,20500,20600,20700,20800,20900,21000,21100,21200,21300,21400,21500,21600,21700,21800,21900,22000,22100,22200,22300,22400,22500,22600,22700,22800,22900,23000,23100,23200,23300,23400,23500,23600,23700,23800,23900,24000,24100,24200,24300,24400,24500,24600,24700,24800,24900,25000,25100,25200,25300,25400,25500,25600,25700,25800,25900,26000,26100,26200,26300,26400,26500,26600,26700,26800,26900,27000,27100,27200,27300,27400,27500,27600,27700,27800,27900,28000,28100,28200,28300,28400,28500,28600,28700,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,29800,29900,30000,30100,30200,30300,30400,30500,30600,30700,30800,30900,31000,31100,31200,31300,31400,31500,31600,31700,31800,31900,32000,32100,32200,32300,32400,32500,32600,32700,32800,32900,33000,33100,33200,33300,33400,33500,33600,33700,33800,33900,34000,34100,34200,34300,34400,34500,34600,34700,34800,34900,35000,35100,35200,35300,35400,35500,35600,35700,35800,35900,36000,36100,36200,36300,36400,36500,36600,36700,36800,36900,37000,37100,37200,37300,37400,37500,37600,37700,37800,37900,38000,38100,38200,38300,38400,38500,38600,38700,38800,38900,39000,39100,39200,39300,39400,39500,39600,39700,39800,39900,40000],"xaxis":"x","y":[0.0,8.41224,16.82448,25.236721,33.64896,42.061195,50.473442,58.885677,67.29792,75.71015,84.12239,92.53463,100.946884,109.35912,117.771355,126.18359,134.59584,143.00807,151.4203,159.83255,168.24478,176.65704,185.06926,193.48152,201.89377,210.30598,218.71825,227.13046,235.54271,243.95494,252.36719,260.77945,269.19168,277.6039,286.01614,294.4284,302.8406,311.25287,319.6651,328.07733,336.48956,344.90186,353.3141,361.7263,370.13852,378.55078,386.96304,395.37527,403.78754,412.19974,420.61197,429.0242,437.4365,445.84872,454.26093,462.6732,471.08542,479.49765,487.90988,496.32217,504.73438,513.1466,521.5589,529.9711,538.38336,546.79553,555.2078,563.62006,572.0323,580.4446,588.8568,597.269,605.6812,614.0935,622.50574,630.91797,639.3302,647.74243,656.15466,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,658.63043,650.21826,641.80597,633.3936,624.98145,616.56915,608.157,599.7448,591.3325,582.9202,574.50806,566.0959,557.6836,549.2713,540.85913,532.4468,524.03467,515.6225,507.21002,498.79785,490.38568,481.9734,473.56122,465.14893,456.7366,448.32446,439.9123,431.49982,423.08765,414.6755,406.26318,397.85104,389.43872,381.0264,372.61426,364.2021,355.78992,347.37747,338.9653,330.55316,322.14084,313.72867,305.31638,296.90405,288.49188,280.07974,271.66742,263.25513,254.84296,246.43063,238.0185,229.60632,221.19385,212.78171,204.36954,195.95721,187.54507,179.13275,170.72046,162.30829,153.89612,145.48364,137.07153,128.65936,120.24719,111.8349,103.42255,95.010376,86.59808,78.18591,69.77374,61.36133,52.949158,44.536987,36.124695,27.712524,19.300232,10.887878,2.475708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"},{"hovertemplate":"\u003cb\u003e%{data.name}\u003c\u002fb\u003e\u003cbr\u003eEmployment income: $%{x:,.0f}\u003cbr\u003eEITC amount: $%{y:,.2f}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Tax Cuts for Workers Act","line":{"color":"#2C6496","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Tax Cuts for Workers Act","orientation":"v","showlegend":true,"x":[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,5000,5100,5200,5300,5400,5500,5600,5700,5800,5900,6000,6100,6200,6300,6400,6500,6600,6700,6800,6900,7000,7100,7200,7300,7400,7500,7600,7700,7800,7900,8000,8100,8200,8300,8400,8500,8600,8700,8800,8900,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900,10000,10100,10200,10300,10400,10500,10600,10700,10800,10900,11000,11100,11200,11300,11400,11500,11600,11700,11800,11900,12000,12100,12200,12300,12400,12500,12600,12700,12800,12900,13000,13100,13200,13300,13400,13500,13600,13700,13800,13900,14000,14100,14200,14300,14400,14500,14600,14700,14800,14900,15000,15100,15200,15300,15400,15500,15600,15700,15800,15900,16000,16100,16200,16300,16400,16500,16600,16700,16800,16900,17000,17100,17200,17300,17400,17500,17600,17700,17800,17900,18000,18100,18200,18300,18400,18500,18600,18700,18800,18900,19000,19100,19200,19300,19400,19500,19600,19700,19800,19900,20100,20200,20300,20400,20500,20600,20700,20800,20900,21000,21100,21200,21300,21400,21500,21600,21700,21800,21900,22000,22100,22200,22300,22400,22500,22600,22700,22800,22900,23000,23100,23200,23300,23400,23500,23600,23700,23800,23900,24000,24100,24200,24300,24400,24500,24600,24700,24800,24900,25000,25100,25200,25300,25400,25500,25600,25700,25800,25900,26000,26100,26200,26300,26400,26500,26600,26700,26800,26900,27000,27100,27200,27300,27400,27500,27600,27700,27800,27900,28000,28100,28200,28300,28400,28500,28600,28700,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,29800,29900,30000,30100,30200,30300,30400,30500,30600,30700,30800,30900,31000,31100,31200,31300,31400,31500,31600,31700,31800,31900,32000,32100,32200,32300,32400,32500,32600,32700,32800,32900,33000,33100,33200,33300,33400,33500,33600,33700,33800,33900,34000,34100,34200,34300,34400,34500,34600,34700,34800,34900,35000,35100,35200,35300,35400,35500,35600,35700,35800,35900,36000,36100,36200,36300,36400,36500,36600,36700,36800,36900,37000,37100,37200,37300,37400,37500,37600,37700,37800,37900,38000,38100,38200,38300,38400,38500,38600,38700,38800,38900,39000,39100,39200,39300,39400,39500,39600,39700,39800,39900,40000],"xaxis":"x","y":[0.0,16.82448,33.64896,50.473442,67.29792,84.12239,100.946884,117.771355,134.59584,151.4203,168.24478,185.06926,201.89377,218.71825,235.54271,252.36719,269.19168,286.01614,302.8406,319.6651,336.48956,353.3141,370.13852,386.96304,403.78754,420.61197,437.4365,454.26093,471.08542,487.90988,504.73438,521.5589,538.38336,555.2078,572.0323,588.8568,605.6812,622.50574,639.3302,656.15466,672.9791,689.8037,706.6282,723.4526,740.27704,757.10156,773.9261,790.75055,807.5751,824.3995,841.22394,858.0484,874.873,891.69745,908.52185,925.3464,942.17084,958.9953,975.81976,992.64435,1009.46875,1026.2932,1043.1178,1059.9423,1076.7667,1093.5911,1110.4156,1127.2401,1144.0646,1160.8892,1177.7136,1194.538,1211.3624,1228.187,1245.0115,1261.8359,1278.6604,1295.4849,1312.3093,1329.1338,1345.9583,1362.7828,1379.6074,1396.4318,1413.2563,1430.0808,1446.9052,1463.7297,1480.5541,1497.3787,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1495.2997,1478.4751,1461.6504,1444.8262,1428.0018,1411.1771,1394.3525,1377.5282,1360.7036,1343.8793,1327.0549,1310.23,1293.4058,1276.5814,1259.7568,1242.9325,1226.1079,1209.2832,1192.4589,1175.6345,1158.8097,1141.9854,1125.161,1108.3364,1091.5121,1074.6875,1057.8628,1041.0386,1024.2141,1007.3899,990.56494,973.7406,956.9163,940.0917,923.26733,906.44275,889.6181,872.79376,855.9695,839.14484,822.32025,805.4959,788.67126,771.847,755.02264,738.1977,721.3734,704.5491,687.7244,670.90015,654.0755,637.2509,620.4266,603.60223,586.77734,569.953,553.12866,536.3044,519.47974,502.6551,485.8308,469.00623,452.1819,435.35754,418.5326,401.70825,384.8839,368.05933,351.235,334.4104,317.58582,300.76147,283.93713,267.11255,250.28784,233.46362,216.63892,199.81458,182.99036,166.1654,149.34106,132.51672,115.69238,98.8678,82.04321,65.21887,48.394287,31.569946,14.745605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Employment income"},"tickformat":"$,.0f"},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"EITC amount"},"tickformat":"$,.0f"},"legend":{"title":{"text":"Scenario"},"tracegroupgap":0,"yanchor":"top","y":0.99,"xanchor":"right","x":0.99,"bgcolor":"rgba(255, 255, 255, 0.8)"},"margin":{"t":50,"l":50,"r":50,"b":100,"pad":4},"font":{"family":"Roboto Serif","color":"#000000"},"title":{"text":"Figure 2: Credit value for married joint filers under current law and the TCWA"},"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

In [156]:
# Create income values from 0 to 40,000 in equal steps
income_steps = 400  # Number of data points
# Generate the original income values
employment_income_original = [
    i * (40000 / (income_steps - 1)) for i in range(income_steps)
]
# Round the income values to the nearest 100
employment_income = [
    round(income / 100) * 100 for income in employment_income_original
]


# Blue Values
blue_values = [
    0.0,
    16.82448,
    33.64896,
    50.473442,
    67.29792,
    84.12239,
    100.946884,
    117.771355,
    134.59584,
    151.4203,
    168.24478,
    185.06926,
    201.89377,
    218.71825,
    235.54271,
    252.36719,
    269.19168,
    286.01614,
    302.8406,
    319.6651,
    336.48956,
    353.3141,
    370.13852,
    386.96304,
    403.78754,
    420.61197,
    437.4365,
    454.26093,
    471.08542,
    487.90988,
    504.73438,
    521.5589,
    538.38336,
    555.2078,
    572.0323,
    588.8568,
    605.6812,
    622.50574,
    639.3302,
    656.15466,
    672.9791,
    689.8037,
    706.6282,
    723.4526,
    740.27704,
    757.10156,
    773.9261,
    790.75055,
    807.5751,
    824.3995,
    841.22394,
    858.0484,
    874.873,
    891.69745,
    908.52185,
    925.3464,
    942.17084,
    958.9953,
    975.81976,
    992.64435,
    1009.46875,
    1026.2932,
    1043.1178,
    1059.9423,
    1076.7667,
    1093.5911,
    1110.4156,
    1127.2401,
    1144.0646,
    1160.8892,
    1177.7136,
    1194.538,
    1211.3624,
    1228.187,
    1245.0115,
    1261.8359,
    1278.6604,
    1295.4849,
    1312.3093,
    1329.1338,
    1345.9583,
    1362.7828,
    1379.6074,
    1396.4318,
    1413.2563,
    1430.0808,
    1446.9052,
    1463.7297,
    1480.5541,
    1497.3787,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1502.0,
    1495.2997,
    1478.4751,
    1461.6504,
    1444.8262,
    1428.0018,
    1411.1771,
    1394.3525,
    1377.5282,
    1360.7036,
    1343.8793,
    1327.0549,
    1310.23,
    1293.4058,
    1276.5814,
    1259.7568,
    1242.9325,
    1226.1079,
    1209.2832,
    1192.4589,
    1175.6345,
    1158.8097,
    1141.9854,
    1125.161,
    1108.3364,
    1091.5121,
    1074.6875,
    1057.8628,
    1041.0386,
    1024.2141,
    1007.3899,
    990.56494,
    973.7406,
    956.9163,
    940.0917,
    923.26733,
    906.44275,
    889.6181,
    872.79376,
    855.9695,
    839.14484,
    822.32025,
    805.4959,
    788.67126,
    771.847,
    755.02264,
    738.1977,
    721.3734,
    704.5491,
    687.7244,
    670.90015,
    654.0755,
    637.2509,
    620.4266,
    603.60223,
    586.77734,
    569.953,
    553.12866,
    536.3044,
    519.47974,
    502.6551,
    485.8308,
    469.00623,
    452.1819,
    435.35754,
    418.5326,
    401.70825,
    384.8839,
    368.05933,
    351.235,
    334.4104,
    317.58582,
    300.76147,
    283.93713,
    267.11255,
    250.28784,
    233.46362,
    216.63892,
    199.81458,
    182.99036,
    166.1654,
    149.34106,
    132.51672,
    115.69238,
    98.8678,
    82.04321,
    65.21887,
    48.394287,
    31.569946,
    14.745605,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]

# Grey Values
grey_values = [
    0.0,
    8.41224,
    16.82448,
    25.236721,
    33.64896,
    42.061195,
    50.473442,
    58.885677,
    67.29792,
    75.71015,
    84.12239,
    92.53463,
    100.946884,
    109.35912,
    117.771355,
    126.18359,
    134.59584,
    143.00807,
    151.4203,
    159.83255,
    168.24478,
    176.65704,
    185.06926,
    193.48152,
    201.89377,
    210.30598,
    218.71825,
    227.13046,
    235.54271,
    243.95494,
    252.36719,
    260.77945,
    269.19168,
    277.6039,
    286.01614,
    294.4284,
    302.8406,
    311.25287,
    319.6651,
    328.07733,
    336.48956,
    344.90186,
    353.3141,
    361.7263,
    370.13852,
    378.55078,
    386.96304,
    395.37527,
    403.78754,
    412.19974,
    420.61197,
    429.0242,
    437.4365,
    445.84872,
    454.26093,
    462.6732,
    471.08542,
    479.49765,
    487.90988,
    496.32217,
    504.73438,
    513.1466,
    521.5589,
    529.9711,
    538.38336,
    546.79553,
    555.2078,
    563.62006,
    572.0323,
    580.4446,
    588.8568,
    597.269,
    605.6812,
    614.0935,
    622.50574,
    630.91797,
    639.3302,
    647.74243,
    656.15466,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    662.0,
    658.63043,
    650.21826,
    641.80597,
    633.3936,
    624.98145,
    616.56915,
    608.157,
    599.7448,
    591.3325,
    582.9202,
    574.50806,
    566.0959,
    557.6836,
    549.2713,
    540.85913,
    532.4468,
    524.03467,
    515.6225,
    507.21002,
    498.79785,
    490.38568,
    481.9734,
    473.56122,
    465.14893,
    456.7366,
    448.32446,
    439.9123,
    431.49982,
    423.08765,
    414.6755,
    406.26318,
    397.85104,
    389.43872,
    381.0264,
    372.61426,
    364.2021,
    355.78992,
    347.37747,
    338.9653,
    330.55316,
    322.14084,
    313.72867,
    305.31638,
    296.90405,
    288.49188,
    280.07974,
    271.66742,
    263.25513,
    254.84296,
    246.43063,
    238.0185,
    229.60632,
    221.19385,
    212.78171,
    204.36954,
    195.95721,
    187.54507,
    179.13275,
    170.72046,
    162.30829,
    153.89612,
    145.48364,
    137.07153,
    128.65936,
    120.24719,
    111.8349,
    103.42255,
    95.010376,
    86.59808,
    78.18591,
    69.77374,
    61.36133,
    52.949158,
    44.536987,
    36.124695,
    27.712524,
    19.300232,
    10.887878,
    2.475708,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]

# Create DataFrame for plotting
df = pd.DataFrame(
    {
        "Employment Income": employment_income,
        "Current Law": grey_values,
        "Tax Cuts for Workers Act": blue_values,
    }
)

# Reshape data for plotting with multiple lines
df_melted = df.melt(
    id_vars="Employment Income",
    value_vars=["Current Law", "Tax Cuts for Workers Act"],
    var_name="Scenario",
    value_name="EITC Amount",
)

# Create the figure
# Create the figure
fig = px.line(
    df_melted,
    x="Employment Income",
    y="EITC Amount",
    color="Scenario",
    color_discrete_map={
        "Current Law": GRAY,
        "Tax Cuts for Workers Act": BLUE_PRIMARY,
    },
    hover_data={
        "Employment Income": ":$,.0f",
        "EITC Amount": ":$,.0f",
    }
).update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_title="Employment income",
    yaxis_title="EITC amount",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat="$,.0f",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 50, "pad": 4},
    title = "Figure 2: Credit value for married joint filers under current law and the TCWA",
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.8)",
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
    images=[
        {
            "source": "/logo512.png",
            "x": 1,
            "y": -0.2,
            "xref": "paper",
            "yref": "paper",
            "sizex": 0.1,
            "sizey": 0.1,
            "xanchor": "right",
            "yanchor": "bottom",
        }
    ],
)

# Update hover template
fig.update_traces(
    hovertemplate="<b>%{data.name}</b><br>" +
                  "Employment income: $%{x:,.0f}<br>" +
                  "EITC amount: $%{y:,.2f}<extra></extra>"
)

# Display the figure
Markdown(

    f"```plotly\n{fig.to_json()}\n```"
)

```plotly
{"data":[{"hovertemplate":"\u003cb\u003e%{data.name}\u003c\u002fb\u003e\u003cbr\u003eEmployment income: $%{x:,.0f}\u003cbr\u003eEITC amount: $%{y:,.2f}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Current Law","line":{"color":"#808080","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Current Law","orientation":"v","showlegend":true,"x":[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,5000,5100,5200,5300,5400,5500,5600,5700,5800,5900,6000,6100,6200,6300,6400,6500,6600,6700,6800,6900,7000,7100,7200,7300,7400,7500,7600,7700,7800,7900,8000,8100,8200,8300,8400,8500,8600,8700,8800,8900,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900,10000,10100,10200,10300,10400,10500,10600,10700,10800,10900,11000,11100,11200,11300,11400,11500,11600,11700,11800,11900,12000,12100,12200,12300,12400,12500,12600,12700,12800,12900,13000,13100,13200,13300,13400,13500,13600,13700,13800,13900,14000,14100,14200,14300,14400,14500,14600,14700,14800,14900,15000,15100,15200,15300,15400,15500,15600,15700,15800,15900,16000,16100,16200,16300,16400,16500,16600,16700,16800,16900,17000,17100,17200,17300,17400,17500,17600,17700,17800,17900,18000,18100,18200,18300,18400,18500,18600,18700,18800,18900,19000,19100,19200,19300,19400,19500,19600,19700,19800,19900,20100,20200,20300,20400,20500,20600,20700,20800,20900,21000,21100,21200,21300,21400,21500,21600,21700,21800,21900,22000,22100,22200,22300,22400,22500,22600,22700,22800,22900,23000,23100,23200,23300,23400,23500,23600,23700,23800,23900,24000,24100,24200,24300,24400,24500,24600,24700,24800,24900,25000,25100,25200,25300,25400,25500,25600,25700,25800,25900,26000,26100,26200,26300,26400,26500,26600,26700,26800,26900,27000,27100,27200,27300,27400,27500,27600,27700,27800,27900,28000,28100,28200,28300,28400,28500,28600,28700,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,29800,29900,30000,30100,30200,30300,30400,30500,30600,30700,30800,30900,31000,31100,31200,31300,31400,31500,31600,31700,31800,31900,32000,32100,32200,32300,32400,32500,32600,32700,32800,32900,33000,33100,33200,33300,33400,33500,33600,33700,33800,33900,34000,34100,34200,34300,34400,34500,34600,34700,34800,34900,35000,35100,35200,35300,35400,35500,35600,35700,35800,35900,36000,36100,36200,36300,36400,36500,36600,36700,36800,36900,37000,37100,37200,37300,37400,37500,37600,37700,37800,37900,38000,38100,38200,38300,38400,38500,38600,38700,38800,38900,39000,39100,39200,39300,39400,39500,39600,39700,39800,39900,40000],"xaxis":"x","y":[0.0,8.41224,16.82448,25.236721,33.64896,42.061195,50.473442,58.885677,67.29792,75.71015,84.12239,92.53463,100.946884,109.35912,117.771355,126.18359,134.59584,143.00807,151.4203,159.83255,168.24478,176.65704,185.06926,193.48152,201.89377,210.30598,218.71825,227.13046,235.54271,243.95494,252.36719,260.77945,269.19168,277.6039,286.01614,294.4284,302.8406,311.25287,319.6651,328.07733,336.48956,344.90186,353.3141,361.7263,370.13852,378.55078,386.96304,395.37527,403.78754,412.19974,420.61197,429.0242,437.4365,445.84872,454.26093,462.6732,471.08542,479.49765,487.90988,496.32217,504.73438,513.1466,521.5589,529.9711,538.38336,546.79553,555.2078,563.62006,572.0323,580.4446,588.8568,597.269,605.6812,614.0935,622.50574,630.91797,639.3302,647.74243,656.15466,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,662.0,658.63043,650.21826,641.80597,633.3936,624.98145,616.56915,608.157,599.7448,591.3325,582.9202,574.50806,566.0959,557.6836,549.2713,540.85913,532.4468,524.03467,515.6225,507.21002,498.79785,490.38568,481.9734,473.56122,465.14893,456.7366,448.32446,439.9123,431.49982,423.08765,414.6755,406.26318,397.85104,389.43872,381.0264,372.61426,364.2021,355.78992,347.37747,338.9653,330.55316,322.14084,313.72867,305.31638,296.90405,288.49188,280.07974,271.66742,263.25513,254.84296,246.43063,238.0185,229.60632,221.19385,212.78171,204.36954,195.95721,187.54507,179.13275,170.72046,162.30829,153.89612,145.48364,137.07153,128.65936,120.24719,111.8349,103.42255,95.010376,86.59808,78.18591,69.77374,61.36133,52.949158,44.536987,36.124695,27.712524,19.300232,10.887878,2.475708,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"},{"hovertemplate":"\u003cb\u003e%{data.name}\u003c\u002fb\u003e\u003cbr\u003eEmployment income: $%{x:,.0f}\u003cbr\u003eEITC amount: $%{y:,.2f}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Tax Cuts for Workers Act","line":{"color":"#2C6496","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Tax Cuts for Workers Act","orientation":"v","showlegend":true,"x":[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,5000,5100,5200,5300,5400,5500,5600,5700,5800,5900,6000,6100,6200,6300,6400,6500,6600,6700,6800,6900,7000,7100,7200,7300,7400,7500,7600,7700,7800,7900,8000,8100,8200,8300,8400,8500,8600,8700,8800,8900,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900,10000,10100,10200,10300,10400,10500,10600,10700,10800,10900,11000,11100,11200,11300,11400,11500,11600,11700,11800,11900,12000,12100,12200,12300,12400,12500,12600,12700,12800,12900,13000,13100,13200,13300,13400,13500,13600,13700,13800,13900,14000,14100,14200,14300,14400,14500,14600,14700,14800,14900,15000,15100,15200,15300,15400,15500,15600,15700,15800,15900,16000,16100,16200,16300,16400,16500,16600,16700,16800,16900,17000,17100,17200,17300,17400,17500,17600,17700,17800,17900,18000,18100,18200,18300,18400,18500,18600,18700,18800,18900,19000,19100,19200,19300,19400,19500,19600,19700,19800,19900,20100,20200,20300,20400,20500,20600,20700,20800,20900,21000,21100,21200,21300,21400,21500,21600,21700,21800,21900,22000,22100,22200,22300,22400,22500,22600,22700,22800,22900,23000,23100,23200,23300,23400,23500,23600,23700,23800,23900,24000,24100,24200,24300,24400,24500,24600,24700,24800,24900,25000,25100,25200,25300,25400,25500,25600,25700,25800,25900,26000,26100,26200,26300,26400,26500,26600,26700,26800,26900,27000,27100,27200,27300,27400,27500,27600,27700,27800,27900,28000,28100,28200,28300,28400,28500,28600,28700,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,29800,29900,30000,30100,30200,30300,30400,30500,30600,30700,30800,30900,31000,31100,31200,31300,31400,31500,31600,31700,31800,31900,32000,32100,32200,32300,32400,32500,32600,32700,32800,32900,33000,33100,33200,33300,33400,33500,33600,33700,33800,33900,34000,34100,34200,34300,34400,34500,34600,34700,34800,34900,35000,35100,35200,35300,35400,35500,35600,35700,35800,35900,36000,36100,36200,36300,36400,36500,36600,36700,36800,36900,37000,37100,37200,37300,37400,37500,37600,37700,37800,37900,38000,38100,38200,38300,38400,38500,38600,38700,38800,38900,39000,39100,39200,39300,39400,39500,39600,39700,39800,39900,40000],"xaxis":"x","y":[0.0,16.82448,33.64896,50.473442,67.29792,84.12239,100.946884,117.771355,134.59584,151.4203,168.24478,185.06926,201.89377,218.71825,235.54271,252.36719,269.19168,286.01614,302.8406,319.6651,336.48956,353.3141,370.13852,386.96304,403.78754,420.61197,437.4365,454.26093,471.08542,487.90988,504.73438,521.5589,538.38336,555.2078,572.0323,588.8568,605.6812,622.50574,639.3302,656.15466,672.9791,689.8037,706.6282,723.4526,740.27704,757.10156,773.9261,790.75055,807.5751,824.3995,841.22394,858.0484,874.873,891.69745,908.52185,925.3464,942.17084,958.9953,975.81976,992.64435,1009.46875,1026.2932,1043.1178,1059.9423,1076.7667,1093.5911,1110.4156,1127.2401,1144.0646,1160.8892,1177.7136,1194.538,1211.3624,1228.187,1245.0115,1261.8359,1278.6604,1295.4849,1312.3093,1329.1338,1345.9583,1362.7828,1379.6074,1396.4318,1413.2563,1430.0808,1446.9052,1463.7297,1480.5541,1497.3787,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1502.0,1495.2997,1478.4751,1461.6504,1444.8262,1428.0018,1411.1771,1394.3525,1377.5282,1360.7036,1343.8793,1327.0549,1310.23,1293.4058,1276.5814,1259.7568,1242.9325,1226.1079,1209.2832,1192.4589,1175.6345,1158.8097,1141.9854,1125.161,1108.3364,1091.5121,1074.6875,1057.8628,1041.0386,1024.2141,1007.3899,990.56494,973.7406,956.9163,940.0917,923.26733,906.44275,889.6181,872.79376,855.9695,839.14484,822.32025,805.4959,788.67126,771.847,755.02264,738.1977,721.3734,704.5491,687.7244,670.90015,654.0755,637.2509,620.4266,603.60223,586.77734,569.953,553.12866,536.3044,519.47974,502.6551,485.8308,469.00623,452.1819,435.35754,418.5326,401.70825,384.8839,368.05933,351.235,334.4104,317.58582,300.76147,283.93713,267.11255,250.28784,233.46362,216.63892,199.81458,182.99036,166.1654,149.34106,132.51672,115.69238,98.8678,82.04321,65.21887,48.394287,31.569946,14.745605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Employment income"},"tickformat":"$,.0f"},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"EITC amount"},"tickformat":"$,.0f"},"legend":{"title":{"text":"Scenario"},"tracegroupgap":0,"yanchor":"top","y":0.99,"xanchor":"right","x":0.99,"bgcolor":"rgba(255, 255, 255, 0.8)"},"margin":{"t":50,"l":50,"r":50,"b":100,"pad":4},"font":{"family":"Roboto Serif","color":"#000000"},"title":{"text":"Figure 2: Credit value for married joint filers under current law and the TCWA"},"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

## Household impacts

Consider a married couple without qualifying children, residing in Colorado and earning $6,000 in 2026. The TCWA increases their net income by [$574](https://policyengine.org/us/household?reform=81471&focus=householdOutput.netIncome&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps&household=52574), due to increased federal and Colorado EITC amounts of $459 and $115 respectively. 

Looking at the [net income distribution](https://policyengine.org/us/household?reform=81471&focus=householdOutput.earnings&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps&household=52574) for this household, we see that the couple will experience an increase in net income until earnings of $29,000, where the federal EITC is fully phased-out under the TCWA. The maximum net income increase occurs between earnings of $10,000 and $19,000 with a spike at $18,000, due to the higher phase-out start.

In [157]:
# Create income values from 0 to 40,000 in equal steps
income_steps = 400  # Number of data points
# Generate the original income values
employment_income_original = [
    i * (40000 / (income_steps - 1)) for i in range(income_steps)
]
# Round the income values to the nearest 100
employment_income = [
    round(income / 100) * 100 for income in employment_income_original
]

# Blue Values from paste.txt
blue_values = [
    0.0,
    10.515625,
    21.030273,
    31.545898,
    42.061523,
    52.57617,
    63.091797,
    73.60742,
    84.12207,
    94.637695,
    105.15381,
    115.66797,
    126.18359,
    136.69824,
    147.21484,
    157.72949,
    168.24414,
    178.76074,
    189.27539,
    199.79004,
    210.30664,
    220.82129,
    231.33594,
    241.85254,
    252.36719,
    262.88184,
    273.39844,
    283.9131,
    294.42773,
    304.94434,
    315.45898,
    325.97363,
    336.49023,
    347.00488,
    357.51953,
    368.03613,
    378.55078,
    389.06543,
    399.58203,
    410.09668,
    420.61133,
    431.1289,
    441.64258,
    452.15723,
    462.67383,
    473.18848,
    483.70312,
    494.21973,
    504.73438,
    515.249,
    525.7656,
    536.2803,
    546.7949,
    557.3115,
    567.8262,
    578.3408,
    588.8574,
    599.3721,
    609.8867,
    620.4033,
    630.91797,
    641.4326,
    651.9492,
    662.46387,
    672.9785,
    683.49414,
    694.00977,
    704.5244,
    715.041,
    725.55566,
    736.0713,
    746.5869,
    757.10156,
    767.6162,
    778.13184,
    788.6465,
    799.1631,
    809.6787,
    820.19336,
    833.917,
    854.94824,
    875.98145,
    897.00684,
    918.0381,
    939.06934,
    960.1006,
    981.1299,
    1002.1621,
    1023.19336,
    1044.2227,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1050.0,
    1054.2109,
    1064.7266,
    1075.2422,
    1085.7578,
    1096.2734,
    1106.7891,
    1117.3047,
    1119.4434,
    1108.9277,
    1098.4121,
    1087.8984,
    1077.3828,
    1066.8652,
    1056.3516,
    1045.8359,
    1035.3203,
    1024.8066,
    1014.291,
    1003.7754,
    993.25977,
    982.74414,
    972.2285,
    961.71484,
    951.1992,
    940.6836,
    930.16797,
    919.65234,
    909.1367,
    898.62305,
    888.1074,
    877.5918,
    867.0762,
    856.56055,
    846.0449,
    835.53125,
    825.0156,
    814.5,
    803.9844,
    793.46875,
    782.9531,
    772.43945,
    761.9238,
    751.40625,
    740.8926,
    730.37695,
    719.8613,
    709.34766,
    698.83203,
    688.3164,
    677.8008,
    667.28516,
    656.76953,
    646.2539,
    635.74023,
    625.22266,
    614.709,
    604.19336,
    593.67773,
    583.16406,
    572.64844,
    562.1328,
    551.61523,
    541.10156,
    530.58594,
    520.07227,
    509.55664,
    499.03906,
    488.5254,
    478.00977,
    467.49414,
    456.98047,
    446.46484,
    435.94922,
    425.4336,
    414.91797,
    404.40234,
    393.88867,
    383.37305,
    372.85742,
    354.92188,
    333.89062,
    312.85938,
    291.82812,
    270.79883,
    249.76562,
    228.73828,
    207.70703,
    186.67578,
    165.64453,
    144.61523,
    123.583984,
    102.55469,
    81.52344,
    60.492188,
    39.46289,
    18.43164,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
]

# Create DataFrame for plotting
df = pd.DataFrame(
    {
        "Employment Income": employment_income,
        "Net income change": blue_values,
    }
)

# Create the figure with Plotly
fig = (
    px.line(
        df,
        x="Employment Income",
        y="Net income change",
        line_shape="linear",
        hover_data={
            "Employment Income": ":$,.0f",
            "Net income change": ":$,.2f",
        }
    )
    .update_traces(
        line=dict(color=BLUE_PRIMARY, width=2.5),
        hovertemplate="Employment Income: $%{x:,.0f}<br>Net income change: $%{y:,.2f}<extra></extra>"
    )
    .update_layout(
        font=dict(family="Roboto Serif"),
        xaxis_title="Employment income",
        yaxis_title="Absolute change in net income",
        xaxis_tickformat="$,.0f",
        yaxis_tickformat="$,.0f",
        font_color=BLACK,
        title="Figure 3: Absolute change in net income under the TCWA",
        margin={"l": 50, "r": 50, "b": 100, "t": 50, "pad": 4},
        annotations=[
            {
                "x": 1,
                "y": -0.25,
                "xref": "paper",
                "yref": "paper",
                "text": "Source: PolicyEngine US",
                "showarrow": False,
                "font": {
                    "family": "Roboto Serif",
                    "size": 10,
                    "color": DARK_GRAY,
                },
            }
        ],
        images=[
            {
                "source": "/logo512.png",
                "x": 1,
                "y": -0.2,
                "xref": "paper",
                "yref": "paper",
                "sizex": 0.1,
                "sizey": 0.1,
                "xanchor": "right",
                "yanchor": "bottom",
            }
        ],
    )
)

# Display the figure
Markdown(f"```plotly\n{fig.to_json()}\n```")

```plotly
{"data":[{"hovertemplate":"Employment Income: $%{x:,.0f}\u003cbr\u003eNet income change: $%{y:,.2f}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","line":{"color":"#2C6496","dash":"solid","shape":"linear","width":2.5},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":[0,100,200,300,400,500,600,700,800,900,1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,5000,5100,5200,5300,5400,5500,5600,5700,5800,5900,6000,6100,6200,6300,6400,6500,6600,6700,6800,6900,7000,7100,7200,7300,7400,7500,7600,7700,7800,7900,8000,8100,8200,8300,8400,8500,8600,8700,8800,8900,9000,9100,9200,9300,9400,9500,9600,9700,9800,9900,10000,10100,10200,10300,10400,10500,10600,10700,10800,10900,11000,11100,11200,11300,11400,11500,11600,11700,11800,11900,12000,12100,12200,12300,12400,12500,12600,12700,12800,12900,13000,13100,13200,13300,13400,13500,13600,13700,13800,13900,14000,14100,14200,14300,14400,14500,14600,14700,14800,14900,15000,15100,15200,15300,15400,15500,15600,15700,15800,15900,16000,16100,16200,16300,16400,16500,16600,16700,16800,16900,17000,17100,17200,17300,17400,17500,17600,17700,17800,17900,18000,18100,18200,18300,18400,18500,18600,18700,18800,18900,19000,19100,19200,19300,19400,19500,19600,19700,19800,19900,20100,20200,20300,20400,20500,20600,20700,20800,20900,21000,21100,21200,21300,21400,21500,21600,21700,21800,21900,22000,22100,22200,22300,22400,22500,22600,22700,22800,22900,23000,23100,23200,23300,23400,23500,23600,23700,23800,23900,24000,24100,24200,24300,24400,24500,24600,24700,24800,24900,25000,25100,25200,25300,25400,25500,25600,25700,25800,25900,26000,26100,26200,26300,26400,26500,26600,26700,26800,26900,27000,27100,27200,27300,27400,27500,27600,27700,27800,27900,28000,28100,28200,28300,28400,28500,28600,28700,28800,28900,29000,29100,29200,29300,29400,29500,29600,29700,29800,29900,30000,30100,30200,30300,30400,30500,30600,30700,30800,30900,31000,31100,31200,31300,31400,31500,31600,31700,31800,31900,32000,32100,32200,32300,32400,32500,32600,32700,32800,32900,33000,33100,33200,33300,33400,33500,33600,33700,33800,33900,34000,34100,34200,34300,34400,34500,34600,34700,34800,34900,35000,35100,35200,35300,35400,35500,35600,35700,35800,35900,36000,36100,36200,36300,36400,36500,36600,36700,36800,36900,37000,37100,37200,37300,37400,37500,37600,37700,37800,37900,38000,38100,38200,38300,38400,38500,38600,38700,38800,38900,39000,39100,39200,39300,39400,39500,39600,39700,39800,39900,40000],"xaxis":"x","y":[0.0,10.515625,21.030273,31.545898,42.061523,52.57617,63.091797,73.60742,84.12207,94.637695,105.15381,115.66797,126.18359,136.69824,147.21484,157.72949,168.24414,178.76074,189.27539,199.79004,210.30664,220.82129,231.33594,241.85254,252.36719,262.88184,273.39844,283.9131,294.42773,304.94434,315.45898,325.97363,336.49023,347.00488,357.51953,368.03613,378.55078,389.06543,399.58203,410.09668,420.61133,431.1289,441.64258,452.15723,462.67383,473.18848,483.70312,494.21973,504.73438,515.249,525.7656,536.2803,546.7949,557.3115,567.8262,578.3408,588.8574,599.3721,609.8867,620.4033,630.91797,641.4326,651.9492,662.46387,672.9785,683.49414,694.00977,704.5244,715.041,725.55566,736.0713,746.5869,757.10156,767.6162,778.13184,788.6465,799.1631,809.6787,820.19336,833.917,854.94824,875.98145,897.00684,918.0381,939.06934,960.1006,981.1299,1002.1621,1023.19336,1044.2227,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1050.0,1054.2109,1064.7266,1075.2422,1085.7578,1096.2734,1106.7891,1117.3047,1119.4434,1108.9277,1098.4121,1087.8984,1077.3828,1066.8652,1056.3516,1045.8359,1035.3203,1024.8066,1014.291,1003.7754,993.25977,982.74414,972.2285,961.71484,951.1992,940.6836,930.16797,919.65234,909.1367,898.62305,888.1074,877.5918,867.0762,856.56055,846.0449,835.53125,825.0156,814.5,803.9844,793.46875,782.9531,772.43945,761.9238,751.40625,740.8926,730.37695,719.8613,709.34766,698.83203,688.3164,677.8008,667.28516,656.76953,646.2539,635.74023,625.22266,614.709,604.19336,593.67773,583.16406,572.64844,562.1328,551.61523,541.10156,530.58594,520.07227,509.55664,499.03906,488.5254,478.00977,467.49414,456.98047,446.46484,435.94922,425.4336,414.91797,404.40234,393.88867,383.37305,372.85742,354.92188,333.89062,312.85938,291.82812,270.79883,249.76562,228.73828,207.70703,186.67578,165.64453,144.61523,123.583984,102.55469,81.52344,60.492188,39.46289,18.43164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Employment income"},"tickformat":"$,.0f"},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Absolute change in net income"},"tickformat":"$,.0f"},"legend":{"tracegroupgap":0},"margin":{"t":50,"l":50,"r":50,"b":100,"pad":4},"font":{"family":"Roboto Serif","color":"#000000"},"title":{"text":"Figure 3: Absolute change in net income under the TCWA"},"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

These changes in the Earned Income Tax Credit [alter marginal tax rates](https://policyengine.org/us/household?reform=81471&focus=householdOutput.mtr&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps&household=52574) from between -18 and +19 percentage points. 

In [158]:
# Create income values from 0 to 40,000 in equal steps
income_steps = 81  # Number of data points
# Generate the original income values
employment_income_original = [
    i * (40000 / (income_steps - 1)) for i in range(income_steps)
]
# Round the income values to the nearest 100
employment_income = [
    round(income / 100) * 100 for income in employment_income_original
]

# Blue Values from paste.txt
blue_values = [
    -0.09562504,
    -0.09562457,
    -0.09562552,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.09562504,
    -0.10727441,
    -0.16038382,
    -0.1780147,
    -0.07179689,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    -0.01118946,
    -0.06429881,
    -0.00807422,
    0.09562492,
    0.09562695,
    0.09562498,
    0.09562504,
    0.09562504,
    0.09562504,
    0.09562504,
    0.09562504,
    0.09562504,
    0.09562504,
    0.09562504,
    0.09562504,
    0.12905663,
    0.18216598,
    0.19124997,
    0.1754961,
    0.06927735,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
]

# Convert blue values to percentages (multiply by 100)
blue_values_percentage = [value * 100 for value in blue_values]

# Create DataFrame for plotting
df = pd.DataFrame(
    {
        "Employment Income": employment_income,
        "Change in marginal tax rate": blue_values,
    }
)

# Create the figure
fig = px.line(
    df,
    x="Employment Income",
    y="Change in marginal tax rate",
    line_shape="linear",
    hover_data={
        "Employment Income": ":$,.0f",
        "Change in marginal tax rate": ":.0f%",
    },
    title = "Figure 4: Change in marginal tax rate by employment income",
).update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_title="Employment income",
    yaxis_title="Change in marginal tax rate",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat=".0%",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 50, "pad": 4},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.8)",
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
    images=[
        {
            "source": "/logo512.png",
            "x": 1,
            "y": -0.2,
            "xref": "paper",
            "yref": "paper",
            "sizex": 0.1,
            "sizey": 0.1,
            "xanchor": "right",
            "yanchor": "bottom",
        }
    ],
)

# Update trace style
fig.update_traces(
    line=dict(color=BLUE_PRIMARY, width=2.5),
    hovertemplate="<b>Change in marginal tax rate</b><br>" +
                  "Employment income: $%{x:,.0f}<br>" +
                  "Blue values percentage: %{customdata:.1f}%<extra></extra>",
    customdata=blue_values_percentage
)

# Display the figure
Markdown(
    f"```plotly\n{fig.to_json()}\n```"
)

```plotly
{"data":[{"hovertemplate":"\u003cb\u003eChange in marginal tax rate\u003c\u002fb\u003e\u003cbr\u003eEmployment income: $%{x:,.0f}\u003cbr\u003eBlue values percentage: %{customdata:.1f}%\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","line":{"color":"#2C6496","dash":"solid","shape":"linear","width":2.5},"marker":{"symbol":"circle"},"mode":"lines","name":"","orientation":"v","showlegend":false,"x":[0,500,1000,1500,2000,2500,3000,3500,4000,4500,5000,5500,6000,6500,7000,7500,8000,8500,9000,9500,10000,10500,11000,11500,12000,12500,13000,13500,14000,14500,15000,15500,16000,16500,17000,17500,18000,18500,19000,19500,20000,20500,21000,21500,22000,22500,23000,23500,24000,24500,25000,25500,26000,26500,27000,27500,28000,28500,29000,29500,30000,30500,31000,31500,32000,32500,33000,33500,34000,34500,35000,35500,36000,36500,37000,37500,38000,38500,39000,39500,40000],"xaxis":"x","y":[-0.09562504,-0.09562457,-0.09562552,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.09562504,-0.10727441,-0.16038382,-0.1780147,-0.07179689,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.01118946,-0.06429881,-0.00807422,0.09562492,0.09562695,0.09562498,0.09562504,0.09562504,0.09562504,0.09562504,0.09562504,0.09562504,0.09562504,0.09562504,0.09562504,0.12905663,0.18216598,0.19124997,0.1754961,0.06927735,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],"yaxis":"y","type":"scatter","customdata":[-9.562503999999999,-9.562457,-9.562552,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-9.562503999999999,-10.727441,-16.038382000000002,-17.80147,-7.179689,0,0,0,0,0,0,0,0,0,0,0,-1.118946,-6.429881,-0.807422,9.562492,9.562695,9.562498,9.562503999999999,9.562503999999999,9.562503999999999,9.562503999999999,9.562503999999999,9.562503999999999,9.562503999999999,9.562503999999999,9.562503999999999,12.905663,18.216598,19.124997,17.549609999999998,6.927735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Employment income"},"tickformat":"$,.0f"},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Change in marginal tax rate"},"tickformat":".0%"},"legend":{"tracegroupgap":0,"yanchor":"top","y":0.99,"xanchor":"right","x":0.99,"bgcolor":"rgba(255, 255, 255, 0.8)"},"title":{"text":"Figure 4: Change in marginal tax rate by employment income"},"font":{"family":"Roboto Serif","color":"#000000"},"margin":{"l":50,"r":50,"b":100,"t":50,"pad":4},"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

This change is reflected in the total marginal tax rate schedule, increasing the marginal rate above 60% for earnings between $19,000 and $20,000, and $27,000 and $27,500, and reducing it to -11% for earnings up to $7,000.

In [159]:
# Create income values from 0 to 40,000 in equal steps
income_steps = 80  # Number of data points
# Generate the original income values
employment_income_original = [
    i * (40000 / (income_steps - 1)) for i in range(income_steps)
]
# Round the income values to the nearest 100
employment_income = [
    round(income / 100) * 100 for income in employment_income_original
]


# Blue Values
blue_values = [
    -0.11475003,
    -0.1147505,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11475003,
    -0.11474907,
    0.02655077,
    0.24138474,
    0.3647051,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.5240508,
    0.627748,
    0.62775004,
    0.56564844,
    0.50625,
    0.50625,
    0.50625,
    0.50625,
    0.50625,
    0.50625,
    0.50625,
    0.50625,
    0.50625,
    0.5089492,
    0.57498825,
    0.6529492,
    0.63719535,
    0.5309785,
    0.371666,
    0.25728709,
    0.22398633,
    0.22049999,
    0.22050196,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22050196,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.43576562,
    0.43576562,
    0.2938633,
    0.2938633,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
]

# Grey Values
grey_values = [
    -0.01912498,
    -0.01912594,
    -0.01912451,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.01912498,
    -0.00747466,
    0.18693459,
    0.41939944,
    0.43650198,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.4365,
    0.44768947,
    0.5007988,
    0.532125,
    0.5321231,
    0.5321231,
    0.47002345,
    0.41062498,
    0.41062498,
    0.41062498,
    0.41062498,
    0.41062498,
    0.41062498,
    0.41062498,
    0.41062498,
    0.3798926,
    0.39282227,
    0.46169925,
    0.46169925,
    0.46170115,
    0.371666,
    0.25728709,
    0.22398633,
    0.22049999,
    0.22049999,
    0.22050196,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22050196,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.43576562,
    0.43576562,
    0.2938633,
    0.2938633,
    0.22049999,
    0.2204999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
    0.22049999,
]

# Convert blue values to percentages (multiply by 100)
blue_values_percentage = [value * 100 for value in blue_values]

grey_values_percentage = [value * 100 for value in grey_values]

# Create DataFrame for plotting
df = pd.DataFrame(
    {
        "Employment Income": employment_income,
        "Current Law": grey_values,
        "Tax Cuts for Workers Act": blue_values,
    }
)

# Reshape data for plotting with multiple lines
df_melted = df.melt(
    id_vars="Employment Income",
    value_vars=["Current Law", "Tax Cuts for Workers Act"],
    var_name="Scenario",
    value_name="Marginal Tax Rate",
)

# Create the figure
fig = px.line(
    df_melted,
    x="Employment Income",
    y="Marginal Tax Rate",
    color="Scenario",
    color_discrete_map={
        "Current Law": GRAY,
        "Tax Cuts for Workers Act": BLUE_PRIMARY,
    },
    title = "Figure 5: Marginal tax rates under current law and the TCWA",
    hover_data={
        "Employment Income": ":$,.0f",
        "Marginal Tax Rate": ":.0%",
    },
).update_layout(
    font=dict(family="Roboto Serif"),
    xaxis_title="Employment income",
    yaxis_title="Marginal Tax Rate",
    xaxis_tickformat="$,.0f",
    yaxis_tickformat=".0%",
    font_color=BLACK,
    margin={"l": 50, "r": 50, "b": 100, "t": 50, "pad": 4},
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.8)",
    ),
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
    images=[
        {
            "source": "/logo512.png",
            "x": 1,
            "y": -0.2,
            "xref": "paper",
            "yref": "paper",
            "sizex": 0.1,
            "sizey": 0.1,
            "xanchor": "right",
            "yanchor": "bottom",
        }
    ],
)
# Display the figure
Markdown(
    f"```plotly\n{fig.to_json()}\n```"
)

```plotly
{"data":[{"hovertemplate":"Scenario=Current Law\u003cbr\u003eEmployment Income=%{x:$,.0f}\u003cbr\u003eMarginal Tax Rate=%{y:.0%}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Current Law","line":{"color":"#808080","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Current Law","orientation":"v","showlegend":true,"x":[0,500,1000,1500,2000,2500,3000,3500,4100,4600,5100,5600,6100,6600,7100,7600,8100,8600,9100,9600,10100,10600,11100,11600,12200,12700,13200,13700,14200,14700,15200,15700,16200,16700,17200,17700,18200,18700,19200,19700,20300,20800,21300,21800,22300,22800,23300,23800,24300,24800,25300,25800,26300,26800,27300,27800,28400,28900,29400,29900,30400,30900,31400,31900,32400,32900,33400,33900,34400,34900,35400,35900,36500,37000,37500,38000,38500,39000,39500,40000],"xaxis":"x","y":[-0.01912498,-0.01912594,-0.01912451,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.01912498,-0.00747466,0.18693459,0.41939944,0.43650198,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.44768947,0.5007988,0.532125,0.5321231,0.5321231,0.47002345,0.41062498,0.41062498,0.41062498,0.41062498,0.41062498,0.41062498,0.41062498,0.41062498,0.3798926,0.39282227,0.46169925,0.46169925,0.46170115,0.371666,0.25728709,0.22398633,0.22049999,0.22049999,0.22050196,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22050196,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.43576562,0.43576562,0.2938633,0.2938633,0.22049999,0.2204999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999],"yaxis":"y","type":"scatter"},{"hovertemplate":"Scenario=Tax Cuts for Workers Act\u003cbr\u003eEmployment Income=%{x:$,.0f}\u003cbr\u003eMarginal Tax Rate=%{y:.0%}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Tax Cuts for Workers Act","line":{"color":"#2C6496","dash":"solid"},"marker":{"symbol":"circle"},"mode":"lines","name":"Tax Cuts for Workers Act","orientation":"v","showlegend":true,"x":[0,500,1000,1500,2000,2500,3000,3500,4100,4600,5100,5600,6100,6600,7100,7600,8100,8600,9100,9600,10100,10600,11100,11600,12200,12700,13200,13700,14200,14700,15200,15700,16200,16700,17200,17700,18200,18700,19200,19700,20300,20800,21300,21800,22300,22800,23300,23800,24300,24800,25300,25800,26300,26800,27300,27800,28400,28900,29400,29900,30400,30900,31400,31900,32400,32900,33400,33900,34400,34900,35400,35900,36500,37000,37500,38000,38500,39000,39500,40000],"xaxis":"x","y":[-0.11475003,-0.1147505,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11475003,-0.11474907,0.02655077,0.24138474,0.3647051,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.4365,0.5240508,0.627748,0.62775004,0.56564844,0.50625,0.50625,0.50625,0.50625,0.50625,0.50625,0.50625,0.50625,0.50625,0.5089492,0.57498825,0.6529492,0.63719535,0.5309785,0.371666,0.25728709,0.22398633,0.22049999,0.22050196,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22050196,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.43576562,0.43576562,0.2938633,0.2938633,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999,0.22049999],"yaxis":"y","type":"scatter"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Employment income"},"tickformat":"$,.0f"},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Marginal Tax Rate"},"tickformat":".0%"},"legend":{"title":{"text":"Scenario"},"tracegroupgap":0,"yanchor":"top","y":0.99,"xanchor":"right","x":0.99,"bgcolor":"rgba(255, 255, 255, 0.8)"},"title":{"text":"Figure 5: Marginal tax rates under current law and the TCWA"},"font":{"family":"Roboto Serif","color":"#000000"},"margin":{"l":50,"r":50,"b":100,"t":50,"pad":4},"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

## National impacts

To estimate the economic effects, we apply the PolicyEngine US 1.247.0 microsimulation model and our [Enhanced Current Population Survey](https://policyengine.org/us/research/enhanced-cps-beta), comparing against current law (assuming TCJA expires). 

### Budgetary impacts

Over the 2026-35 budget window, and assuming no behavioral responses, the reform would cost the federal government $65.4 billion, with costs decreasing from [$7.0 billion](https://policyengine.org/us/policy?reform=81471&focus=policyOutput.budgetaryImpact.overall&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps) in 2026 to $6.1 billion in 2035. Applying labor supply elasticities from the Congressional Budget Office, the federal cost rises 168.8% to $175.8 billion over the budget window. For example, applying CBO elasticities in 2026, we project that the reform [reduces earning](https://policyengine.org/us/policy?reform=81478&focus=policyOutput.laborSupplyImpact.earnings.overall.relative&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps), which in turn reduces tax revenues and increases outlays for programs like SNAP.


In [160]:
# Your original data
years = [2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035]
static_impact = [7.0, 6.8, 6.7, 6.7, 6.6, 6.5, 6.4, 6.3, 6.2, 6.1]
dynamic_impact = [19.2, 19.0, 18.4, 18.6, 18.2, 17.5, 16.8, 16.5, 16.1, 15.5]

# Create DataFrame from your data
df = pd.DataFrame(
    {
        "Year": years,
        "Static ": static_impact,
        "With CBO behavioral responses": dynamic_impact,
    }
)

# Import tabulate directly
from tabulate import tabulate

# Convert DataFrame to a list of lists for tabulate
headers = df.columns.tolist()
data = df.values.tolist()

# Create a list of format strings for each column
# First column (Year) as integer, others with one decimal place
colalign = ["right"] * len(headers)  # Right-align all columns
floatfmt = [".0f", ".1f", ".1f"]  # Format for each column

# Generate the markdown table
markdown_table = tabulate(
    data,
    headers=headers,
    tablefmt="pipe",
    colalign=colalign,
    floatfmt=floatfmt,
)

# Display the markdown table
Markdown(
    "**Table 2: Budget Window Impact of the Tax Cuts for Workers Act, with and without behavioral responses.**\n\n"
    + markdown_table
)

**Table 2: Budget Window Impact of the Tax Cuts for Workers Act, with and without behavioral responses.**

|   Year |   Static  |   With CBO behavioral responses |
|-------:|----------:|--------------------------------:|
|   2026 |       7.0 |                            19.2 |
|   2027 |       6.8 |                            19.0 |
|   2028 |       6.7 |                            18.4 |
|   2029 |       6.7 |                            18.6 |
|   2030 |       6.6 |                            18.2 |
|   2031 |       6.5 |                            17.5 |
|   2032 |       6.4 |                            16.8 |
|   2033 |       6.3 |                            16.5 |
|   2034 |       6.2 |                            16.1 |
|   2035 |       6.1 |                            15.5 |

## Distributional impacts

Here we report various distributional impacts for 2026. These account for mechanical state and local tax impacts due to EITC matches, which we project at $1.5 billion for 2026 (in addition to the $7.0 billion federal cost).

The TCWA would [increase the net income](https://policyengine.org/us/policy?reform=81471&focus=policyOutput.winnersAndLosers.incomeDecile&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps) of 10% of the population, with those in the eighth decile most likely to experience an increase in household income (26%). Nobody’s net income falls. We assume no behavioral responses.

In [161]:
# Income deciles
deciles = list(range(1, 11))

# Data for tax cut effects
gain_more_than_5pct = [9, 5, 4, 1.1, 0, 0, 0, 0, 0, 0]  # Dark blue
gain_less_than_5pct = [6, 11, 9, 11, 5, 11, 7, 10, 1.4, 8]  # Light blue
no_change = [85, 84, 87, 88, 95, 88, 93, 90, 98.6, 92]  # Neutral grey
loss_less_than_5pct = [
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
]  # Light red (empty in this case)
loss_more_than_5pct = [
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
]  # Dark red (empty in this case)

# Direct input values for the "All" category
all_gain_more = 2
all_gain_less = 7.8
all_no_change = 91.2
all_loss_less = 0
all_loss_more = 0

# Create labels for deciles
labels_deciles = [f"{i}" for i in deciles]

# Create DataFrame for deciles
df_deciles = pd.DataFrame(
    {
        "Income decile": labels_deciles,
        "Gain more than 5%": gain_more_than_5pct,
        "Gain less than 5%": gain_less_than_5pct,
        "No change": no_change,
        "Loss less than 5%": loss_less_than_5pct,
        "Loss more than 5%": loss_more_than_5pct,
    }
)

# Create DataFrame for All
df_all = pd.DataFrame(
    {
        "Income decile": ["All"],
        "Gain more than 5%": [all_gain_more],
        "Gain less than 5%": [all_gain_less],
        "No change": [all_no_change],
        "Loss less than 5%": [all_loss_less],
        "Loss more than 5%": [all_loss_more],
    }
)

# Create the final combined figure with a subplot
fig = make_subplots(
    rows=2,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    row_heights=[0.1, 0.9],
)

# Add traces for "All" category - first row
fig.add_trace(
    go.Bar(
        y=df_all["Income decile"],
        x=df_all["Gain more than 5%"],
        name="Gain more than 5%",
        orientation="h",
        marker_color=BLUE_PRIMARY,
        text=[f"{x}%" if x > 0 else "" for x in df_all["Gain more than 5%"]],
        textposition="inside",
        legendgroup="Gain more than 5%",
        showlegend=True,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_all["Income decile"],
        x=df_all["Gain less than 5%"],
        name="Gain less than 5%",
        orientation="h",
        marker_color=BLUE_LIGHT,
        text=[f"{x}%" if x > 0 else "" for x in df_all["Gain less than 5%"]],
        textposition="inside",
        legendgroup="Gain less than 5%",
        showlegend=True,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_all["Income decile"],
        x=df_all["No change"],
        name="No change",
        orientation="h",
        marker_color=LIGHT_GRAY,
        text=[f"{x}%" if x > 0 else "" for x in df_all["No change"]],
        textposition="inside",
        legendgroup="No change",
        showlegend=True,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_all["Income decile"],
        x=df_all["Loss less than 5%"],
        name="Loss less than 5%",
        orientation="h",
        marker_color=MEDIUM_DARK_GRAY,
        text=[f"{x}%" if x > 0 else "" for x in df_all["Loss less than 5%"]],
        textposition="inside",
        legendgroup="Loss less than 5%",
        showlegend=True,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_all["Income decile"],
        x=df_all["Loss more than 5%"],
        name="Loss more than 5%",
        orientation="h",
        marker_color=DARK_GRAY,
        text=[f"{x}%" if x > 0 else "" for x in df_all["Loss more than 5%"]],
        textposition="inside",
        legendgroup="Loss more than 5%",
        showlegend=True,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=1,
    col=1,
)

# Add traces for deciles - second row
fig.add_trace(
    go.Bar(
        y=df_deciles["Income decile"],
        x=df_deciles["Gain more than 5%"],
        name="Gain more than 5%",
        orientation="h",
        marker_color=BLUE_PRIMARY,
        text=[
            f"{x}%" if x > 0 else "" for x in df_deciles["Gain more than 5%"]
        ],
        textposition="inside",
        legendgroup="Gain more than 5%",
        showlegend=False,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_deciles["Income decile"],
        x=df_deciles["Gain less than 5%"],
        name="Gain less than 5%",
        orientation="h",
        marker_color=BLUE_LIGHT,
        text=[
            f"{x}%" if x > 0 else "" for x in df_deciles["Gain less than 5%"]
        ],
        textposition="inside",
        legendgroup="Gain less than 5%",
        showlegend=False,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_deciles["Income decile"],
        x=df_deciles["No change"],
        name="No change",
        orientation="h",
        marker_color=LIGHT_GRAY,
        text=[f"{x}%" if x > 0 else "" for x in df_deciles["No change"]],
        textposition="inside",
        legendgroup="No change",
        showlegend=False,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_deciles["Income decile"],
        x=df_deciles["Loss less than 5%"],
        name="Loss less than 5%",
        orientation="h",
        marker_color=MEDIUM_DARK_GRAY,
        text=[
            f"{x}%" if x > 0 else "" for x in df_deciles["Loss less than 5%"]
        ],
        textposition="inside",
        legendgroup="Loss less than 5%",
        showlegend=False,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Bar(
        y=df_deciles["Income decile"],
        x=df_deciles["Loss more than 5%"],
        name="Loss more than 5%",
        orientation="h",
        marker_color=DARK_GRAY,
        text=[
            f"{x}%" if x > 0 else "" for x in df_deciles["Loss more than 5%"]
        ],
        textposition="inside",
        legendgroup="Loss more than 5%",
        showlegend=False,
        hovertemplate="%{x}<extra></extra>",
    ),
    row=2,
    col=1,
)

# Update layout
fig.update_layout(
    barmode="stack",
    title_x=0.8,
    font=dict(family="Roboto Serif"),
    title="Figure 6: Winners of the Tax Cuts for Workers Act by Income Decile",
    xaxis=dict(title="", ticksuffix="%", range=[0, 100]),
    xaxis2=dict(
        title=dict(
            text="Population share",
            standoff=20,  # Increase standoff to move title down
        ),
        ticksuffix="%",
        range=[0, 100],
    ),
    yaxis2=dict(
        automargin=True,
        title=dict(
            text="Income Decile",
            standoff=15,  # Increase standoff to move title left
        ),
    ),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1.0,
        xanchor="right",
        x=1.5,
        traceorder="normal",
        itemsizing="constant",
    ),
    font_color=BLACK,
    margin={"l": 50, "r": 160, "b": 100, "t": 100, "pad": 4},
    height=600,
    width=900,  # Explicitly set width to accommodate legend
    annotations=[
        {
            "x": 1,
            "y": -0.25,
            "xref": "paper",
            "yref": "paper",
            "text": "Source: PolicyEngine US",
            "showarrow": False,
            "font": {"family": "Roboto Serif", "size": 10, "color": DARK_GRAY},
        }
    ],
    images=[
        {
            "source": "/logo512.png",
            "x": 1,
            "y": -0.2,
            "xref": "paper",
            "yref": "paper",
            "sizex": 0.1,
            "sizey": 0.1,
            "xanchor": "right",
            "yanchor": "bottom",
        }
    ],
)

# Display the figure
Markdown(
    f"```plotly\n{fig.to_json()}\n```"
)

```plotly
{"data":[{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Gain more than 5%","marker":{"color":"#2C6496"},"name":"Gain more than 5%","orientation":"h","showlegend":true,"text":["2%"],"textposition":"inside","x":[2],"y":["All"],"type":"bar","xaxis":"x","yaxis":"y"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Gain less than 5%","marker":{"color":"#D8E6F3"},"name":"Gain less than 5%","orientation":"h","showlegend":true,"text":["7.8%"],"textposition":"inside","x":[7.8],"y":["All"],"type":"bar","xaxis":"x","yaxis":"y"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"No change","marker":{"color":"#F2F2F2"},"name":"No change","orientation":"h","showlegend":true,"text":["91.2%"],"textposition":"inside","x":[91.2],"y":["All"],"type":"bar","xaxis":"x","yaxis":"y"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loss less than 5%","marker":{"color":"#D2D2D2"},"name":"Loss less than 5%","orientation":"h","showlegend":true,"text":[""],"textposition":"inside","x":[0],"y":["All"],"type":"bar","xaxis":"x","yaxis":"y"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loss more than 5%","marker":{"color":"#616161"},"name":"Loss more than 5%","orientation":"h","showlegend":true,"text":[""],"textposition":"inside","x":[0],"y":["All"],"type":"bar","xaxis":"x","yaxis":"y"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Gain more than 5%","marker":{"color":"#2C6496"},"name":"Gain more than 5%","orientation":"h","showlegend":false,"text":["9.0%","5.0%","4.0%","1.1%","","","","","",""],"textposition":"inside","x":[9.0,5.0,4.0,1.1,0.0,0.0,0.0,0.0,0.0,0.0],"y":["1","2","3","4","5","6","7","8","9","10"],"type":"bar","xaxis":"x2","yaxis":"y2"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Gain less than 5%","marker":{"color":"#D8E6F3"},"name":"Gain less than 5%","orientation":"h","showlegend":false,"text":["6.0%","11.0%","9.0%","11.0%","5.0%","11.0%","7.0%","10.0%","1.4%","8.0%"],"textposition":"inside","x":[6.0,11.0,9.0,11.0,5.0,11.0,7.0,10.0,1.4,8.0],"y":["1","2","3","4","5","6","7","8","9","10"],"type":"bar","xaxis":"x2","yaxis":"y2"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"No change","marker":{"color":"#F2F2F2"},"name":"No change","orientation":"h","showlegend":false,"text":["85.0%","84.0%","87.0%","88.0%","95.0%","88.0%","93.0%","90.0%","98.6%","92.0%"],"textposition":"inside","x":[85.0,84.0,87.0,88.0,95.0,88.0,93.0,90.0,98.6,92.0],"y":["1","2","3","4","5","6","7","8","9","10"],"type":"bar","xaxis":"x2","yaxis":"y2"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loss less than 5%","marker":{"color":"#D2D2D2"},"name":"Loss less than 5%","orientation":"h","showlegend":false,"text":["","","","","","","","","",""],"textposition":"inside","x":[0,0,0,0,0,0,0,0,0,0],"y":["1","2","3","4","5","6","7","8","9","10"],"type":"bar","xaxis":"x2","yaxis":"y2"},{"hovertemplate":"%{x}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"Loss more than 5%","marker":{"color":"#616161"},"name":"Loss more than 5%","orientation":"h","showlegend":false,"text":["","","","","","","","","",""],"textposition":"inside","x":[0,0,0,0,0,0,0,0,0,0],"y":["1","2","3","4","5","6","7","8","9","10"],"type":"bar","xaxis":"x2","yaxis":"y2"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"matches":"x2","showticklabels":false,"title":{"text":""},"ticksuffix":"%","range":[0,100]},"yaxis":{"anchor":"x","domain":[0.902,1.0]},"xaxis2":{"anchor":"y2","domain":[0.0,1.0],"title":{"text":"Population share","standoff":20},"ticksuffix":"%","range":[0,100]},"yaxis2":{"anchor":"x2","domain":[0.0,0.882],"title":{"text":"Income Decile","standoff":15},"automargin":true},"title":{"text":"Figure 6: Winners of the Tax Cuts for Workers Act by Income Decile"},"font":{"family":"Roboto Serif","color":"#000000"},"legend":{"orientation":"v","yanchor":"top","y":1.0,"xanchor":"right","x":1.5,"traceorder":"normal","itemsizing":"constant"},"margin":{"l":50,"r":160,"b":100,"t":100,"pad":4},"barmode":"stack","height":600,"width":900,"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":1,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

The TCWA would increase the [net income of households by $59](https://policyengine.org/us/policy?reform=81471&focus=policyOutput.distributionalImpact.incomeDecile.average&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps), with the largest increase in the first ($95) and second income deciles ($91). Households in upper income deciles still benefit as they can include multiple tax units.


In [162]:
# Average impact by decile data
deciles = list(range(1, 11))
avg_impact = [95, 91, 79, 59, 27, 54, 41, 50, 8, 55]  # Values in dollars

df = pd.DataFrame(
    {
        "Income decile": deciles,
        "Average impact": avg_impact,
    }
)

# Format dollar values for display
dollar_text = [f"${x}" for x in avg_impact]

fig = (
    px.bar(
        df,
        x="Income decile",
        y="Average impact",
        text=dollar_text,  # Dollar values with $ sign
        color_discrete_sequence=[BLUE_PRIMARY],
        title="Average Impact of the Tax Cuts for Workers Act by Income Decile",
    )
    .update_layout(
        font=dict(family="Roboto Serif"),
        xaxis_title="Income decile",
        yaxis_title="Average impact",
        xaxis_tickvals=list(range(1, 11)),
        yaxis_tickformat=",",
        showlegend=False,
        font_color=BLACK,
        margin={"l": 50, "r": 50, "b": 100, "t": 50, "pad": 4},
        title = "Figure 7: Average impact of the Tax Cuts for Workers Act by income decile",
        annotations=[
            {
                "x": 0.5,
                "y": -0.25,
                "xref": "paper",
                "yref": "paper",
                "text": "Source: PolicyEngine US",
                "showarrow": False,
                "font": {
                    "family": "Roboto Serif",
                    "size": 10,
                    "color": DARK_GRAY,
                },
            }
        ],
        images=[
            {
                "source": "/logo512.png",
                "x": 1,
                "y": -0.2,
                "xref": "paper",
                "yref": "paper",
                "sizex": 0.1,
                "sizey": 0.1,
                "xanchor": "right",
                "yanchor": "bottom",
            }
        ],
    )
    .update_traces(
        hovertemplate="Income decile: %{x}<br>Average impact: $%{y:,.0f}<extra></extra>"
    )
)

Markdown(
    f"```plotly\n{fig.to_json()}\n```"
)

```plotly
{"data":[{"alignmentgroup":"True","hovertemplate":"Income decile: %{x}\u003cbr\u003eAverage impact: $%{y:,.0f}\u003cextra\u003e\u003c\u002fextra\u003e","legendgroup":"","marker":{"color":"#2C6496","pattern":{"shape":""}},"name":"","offsetgroup":"","orientation":"v","showlegend":false,"text":["$95","$91","$79","$59","$27","$54","$41","$50","$8","$55"],"textposition":"auto","x":[1,2,3,4,5,6,7,8,9,10],"xaxis":"x","y":[95,91,79,59,27,54,41,50,8,55],"yaxis":"y","type":"bar"}],"layout":{"template":{"data":{"histogram2dcontour":[{"type":"histogram2dcontour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"choropleth":[{"type":"choropleth","colorbar":{"outlinewidth":0,"ticks":""}}],"histogram2d":[{"type":"histogram2d","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmap":[{"type":"heatmap","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"heatmapgl":[{"type":"heatmapgl","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"contourcarpet":[{"type":"contourcarpet","colorbar":{"outlinewidth":0,"ticks":""}}],"contour":[{"type":"contour","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"surface":[{"type":"surface","colorbar":{"outlinewidth":0,"ticks":""},"colorscale":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]]}],"mesh3d":[{"type":"mesh3d","colorbar":{"outlinewidth":0,"ticks":""}}],"scatter":[{"fillpattern":{"fillmode":"overlay","size":10,"solidity":0.2},"type":"scatter"}],"parcoords":[{"type":"parcoords","line":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolargl":[{"type":"scatterpolargl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"bar":[{"error_x":{"color":"#2a3f5f"},"error_y":{"color":"#2a3f5f"},"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"bar"}],"scattergeo":[{"type":"scattergeo","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterpolar":[{"type":"scatterpolar","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"histogram":[{"marker":{"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"histogram"}],"scattergl":[{"type":"scattergl","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatter3d":[{"type":"scatter3d","line":{"colorbar":{"outlinewidth":0,"ticks":""}},"marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattermapbox":[{"type":"scattermapbox","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scatterternary":[{"type":"scatterternary","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"scattercarpet":[{"type":"scattercarpet","marker":{"colorbar":{"outlinewidth":0,"ticks":""}}}],"carpet":[{"aaxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"baxis":{"endlinecolor":"#2a3f5f","gridcolor":"white","linecolor":"white","minorgridcolor":"white","startlinecolor":"#2a3f5f"},"type":"carpet"}],"table":[{"cells":{"fill":{"color":"#EBF0F8"},"line":{"color":"white"}},"header":{"fill":{"color":"#C8D4E3"},"line":{"color":"white"}},"type":"table"}],"barpolar":[{"marker":{"line":{"color":"#E5ECF6","width":0.5},"pattern":{"fillmode":"overlay","size":10,"solidity":0.2}},"type":"barpolar"}],"pie":[{"automargin":true,"type":"pie"}]},"layout":{"autotypenumbers":"strict","colorway":["#636efa","#EF553B","#00cc96","#ab63fa","#FFA15A","#19d3f3","#FF6692","#B6E880","#FF97FF","#FECB52"],"font":{"color":"#2a3f5f"},"hovermode":"closest","hoverlabel":{"align":"left"},"paper_bgcolor":"white","plot_bgcolor":"#E5ECF6","polar":{"bgcolor":"#E5ECF6","angularaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"radialaxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"ternary":{"bgcolor":"#E5ECF6","aaxis":{"gridcolor":"white","linecolor":"white","ticks":""},"baxis":{"gridcolor":"white","linecolor":"white","ticks":""},"caxis":{"gridcolor":"white","linecolor":"white","ticks":""}},"coloraxis":{"colorbar":{"outlinewidth":0,"ticks":""}},"colorscale":{"sequential":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"sequentialminus":[[0.0,"#0d0887"],[0.1111111111111111,"#46039f"],[0.2222222222222222,"#7201a8"],[0.3333333333333333,"#9c179e"],[0.4444444444444444,"#bd3786"],[0.5555555555555556,"#d8576b"],[0.6666666666666666,"#ed7953"],[0.7777777777777778,"#fb9f3a"],[0.8888888888888888,"#fdca26"],[1.0,"#f0f921"]],"diverging":[[0,"#8e0152"],[0.1,"#c51b7d"],[0.2,"#de77ae"],[0.3,"#f1b6da"],[0.4,"#fde0ef"],[0.5,"#f7f7f7"],[0.6,"#e6f5d0"],[0.7,"#b8e186"],[0.8,"#7fbc41"],[0.9,"#4d9221"],[1,"#276419"]]},"xaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"yaxis":{"gridcolor":"white","linecolor":"white","ticks":"","title":{"standoff":15},"zerolinecolor":"white","automargin":true,"zerolinewidth":2},"scene":{"xaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"yaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2},"zaxis":{"backgroundcolor":"#E5ECF6","gridcolor":"white","linecolor":"white","showbackground":true,"ticks":"","zerolinecolor":"white","gridwidth":2}},"shapedefaults":{"line":{"color":"#2a3f5f"}},"annotationdefaults":{"arrowcolor":"#2a3f5f","arrowhead":0,"arrowwidth":1},"geo":{"bgcolor":"white","landcolor":"#E5ECF6","subunitcolor":"white","showland":true,"showlakes":true,"lakecolor":"white"},"title":{"x":0.05},"mapbox":{"style":"light"}}},"xaxis":{"anchor":"y","domain":[0.0,1.0],"title":{"text":"Income decile"},"tickvals":[1,2,3,4,5,6,7,8,9,10]},"yaxis":{"anchor":"x","domain":[0.0,1.0],"title":{"text":"Average impact"},"tickformat":","},"legend":{"tracegroupgap":0},"title":{"text":"Figure 7: Average impact of the Tax Cuts for Workers Act by income decile"},"barmode":"relative","font":{"family":"Roboto Serif","color":"#000000"},"margin":{"l":50,"r":50,"b":100,"t":50,"pad":4},"showlegend":false,"annotations":[{"font":{"color":"#616161","family":"Roboto Serif","size":10},"showarrow":false,"text":"Source: PolicyEngine US","x":0.5,"xref":"paper","y":-0.25,"yref":"paper"}],"images":[{"sizex":0.1,"sizey":0.1,"source":"\u002flogo512.png","x":1,"xanchor":"right","xref":"paper","y":-0.2,"yanchor":"bottom","yref":"paper"}]}}
```

### Poverty and inequality impacts

Our analysis finds that the TCWA would [reduce the poverty rate](https://policyengine.org/us/policy?reform=81471&focus=policyOutput.povertyImpact.regular.byAge&region=us&timePeriod=2026&baseline=2&dataset=enhanced_cps) by 0.3% (0.1 percentage point reduction in the Supplemental Poverty Measure) in 2026, disproportionately among adults aged 18-64 (0.4%).

## Conclusion


The Tax Cuts for Workers Act would cost the federal government $65.4 billion over the 2026-35 budget window, assuming no behavioral responses. Accounting for state and local tax impacts, which add 21% to the aggregate impact beyond the federal taxes, it increases net income for 10% of the population, with an average household increase of $59 ($590 per affected household). Households in the first and second income deciles would see the largest benefits ($95 and $91 respectively). The TCWA is projected to reduce poverty by 0.3% and the Gini index of income inequality by 0.05%. While it reduces marginal tax rates for the lowest earners, substitution effects from the higher marginal tax rates in the phase-out range combine with income effects to reduce labor supply and increase costs when applying CBO’s labor supply elasticities.

We invite readers to explore this reform and others in the [PolicyEngine app](http://policyengine.org).
